In [1]:
import pandas as pd
import numpy as np
import sys
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
import joblib
import re

In [2]:
data = pd.read_csv('data.csv', encoding = "ISO-8859-1", names=['sentiment', 'id', 'date_time', 'flag', 'username','tweets'])
data

,sentiment,id,date_time,flag,username,tweets
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


# Data preprocesing


In [3]:
data = data.drop(['id', 'date_time', "flag", 'username'], axis=1)
data

,sentiment,tweets
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...


In [4]:
#function to remove punctuations and links as they are irrelevant to sentiments
def remove_pun_links(tweets):
    texts = np.array(tweets)
    import string
    for i in range(len(texts)):
        texts[i] = re.sub(r'http\S+','', texts[i])
        texts[i] = re.sub(r'\S+.com','', texts[i])        
        texts[i] = texts[i].translate(str.maketrans('', '', string.punctuation))
    return texts
#removing punctuations
pun_removed = remove_pun_links(data["tweets"].to_numpy())
pd.DataFrame(pun_removed)

,0
0,switchfoot Awww thats a bummer You shoulda ...
1,is upset that he cant update his Facebook by t...
2,Kenichan I dived many times for the ball Manag...
3,my whole body feels itchy and like its on fire
4,nationwideclass no its not behaving at all im ...
...,...
1599995,Just woke up Having no school is the best feel...
1599996,Very cool to hear old Walt interviews â«
1599997,Are you ready for your MoJo Makeover Ask me fo...
1599998,Happy 38th Birthday to my boo of alll time Tup...


In [5]:
#function to slit the words in each tweet
def word_split(tweets):
    texts = np.array(tweets)
    print(len(texts))
    for i in range(len(texts)):
        texts[i] = texts[i].split()
    return texts
words = word_split(pun_removed)


1600000


In [6]:
#changing the elements of words to np array
for i in range(len(words)):
    words[i] = np.array(words[i])
type(words[0])

numpy.ndarray

In [7]:
#function to dictionarize two np arrays
def dictionarize_freq(texts):
    word_count = []
    for i in range(len(words)):
        dic = {}
        keys, values = np.unique(words[i], return_counts=True)
        for k, v in (zip(keys,values)):
            dic[k] = v
#         for j in range(len(keys)):
#             dic[keys[j]] = values[j]
        word_count.append(dic)
    return word_count

In [8]:
data_word_count = np.array(dictionarize_freq(words))
data['word_count'] = data_word_count
data

,sentiment,tweets,word_count
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","{'Awww': 1, 'Carr': 1, 'D': 1, 'David': 1, 'Da..."
1,0,is upset that he can't update his Facebook by ...,"{'Blah': 1, 'Facebook': 1, 'School': 1, 'a': 1..."
2,0,@Kenichan I dived many times for the ball. Man...,"{'50': 1, 'I': 1, 'Kenichan': 1, 'Managed': 1,..."
3,0,my whole body feels itchy and like its on fire,"{'and': 1, 'body': 1, 'feels': 1, 'fire': 1, '..."
4,0,"@nationwideclass no, it's not behaving at all....","{'I': 1, 'all': 2, 'am': 1, 'at': 1, 'because'..."
...,...,...,...
1599995,4,Just woke up. Having no school is the best fee...,"{'Having': 1, 'Just': 1, 'best': 1, 'ever': 1,..."
1599996,4,TheWDB.com - Very cool to hear old Walt interv...,"{'Very': 1, 'Walt': 1, 'cool': 1, 'hear': 1, '..."
1599997,4,Are you ready for your MoJo Makeover? Ask me f...,"{'Are': 1, 'Ask': 1, 'Makeover': 1, 'MoJo': 1,..."
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...,"{'38th': 1, 'Amaru': 1, 'Birthday': 1, 'Happy'..."


In [12]:

#splitting of training and tesing data
vectorizer = DictVectorizer()
x = vectorizer.fit_transform(list(data['word_count']))
x

<1600000x899027 sparse matrix of type '<class 'numpy.float64'>'
	with 19742256 stored elements in Compressed Sparse Row format>

In [13]:
print(x)

  (0, 37114)	1.0
  (0, 63488)	1.0
  (0, 77492)	1.0
  (0, 84045)	1.0
  (0, 84477)	1.0
  (0, 289837)	1.0
  (0, 319644)	1.0
  (0, 322385)	1.0
  (0, 390611)	1.0
  (0, 449175)	1.0
  (0, 507623)	1.0
  (0, 550260)	1.0
  (0, 673950)	1.0
  (0, 770690)	1.0
  (0, 805323)	1.0
  (0, 815389)	1.0
  (0, 828060)	1.0
  (1, 48993)	1.0
  (1, 105176)	1.0
  (1, 258794)	1.0
  (1, 322385)	1.0
  (1, 337860)	1.0
  (1, 342393)	1.0
  (1, 351775)	1.0
  (1, 392781)	1.0
  :	:
  (1599997, 185989)	1.0
  (1599997, 198595)	1.0
  (1599997, 442263)	1.0
  (1599997, 488763)	2.0
  (1599997, 626860)	1.0
  (1599997, 733189)	1.0
  (1599997, 887447)	1.0
  (1599997, 888216)	1.0
  (1599998, 10283)	1.0
  (1599998, 28813)	1.0
  (1599998, 48362)	1.0
  (1599998, 128026)	1.0
  (1599998, 261766)	1.0
  (1599998, 296163)	1.0
  (1599998, 336500)	1.0
  (1599998, 380667)	1.0
  (1599998, 653379)	1.0
  (1599998, 673950)	1.0
  (1599998, 825083)	1.0
  (1599998, 828060)	1.0
  (1599999, 270847)	1.0
  (1599999, 270964)	1.0
  (1599999, 404037)	1.0
 

# Applying Logistic Regression

In [34]:
#splitting of training and tesing data
# vectorizer = DictVectorizer()
# x = vectorizer.fit_transform(word_x)

# x = data['word_count']
y = data['sentiment']
x_train, x_test, y_train, y_test = train_test_split(x,y)

In [32]:
#Creating the model
model = LogisticRegression(max_iter=1000000)
model.fit(x_train,y_train)

LogisticRegression(max_iter=1000000)

In [35]:
print(y_test)

from sklearn import metrics
print(metrics.confusion_matrix(y_test, model.predict(x_test)))
model.score(x_test, y_test)

437936     0
684417     0
1464635    4
728379     0
368788     0
          ..
614077     0
168113     0
627404     0
56421      0
143939     0
Name: sentiment, Length: 400000, dtype: int64
[[168720  31126]
 [ 27723 172431]]


0.8528775

# Saving the model and vectorizer


In [36]:
joblib.dump(model,'model')


['model']

In [37]:
%store vectorizer

Stored 'vectorizer' (DictVectorizer)
